---
# <span style="font-size:25px;"> **Project:** Thermal error compensation
#  <span style="font-size:25px;"> **Machine:** 25TEX (SJE-10)
#  <span style="font-size:25px;"> **Phase:** P04 (Ansys simulation data)
# <span style="font-size:25px;"> **Activity:** Cross-validation for 80% training and 20% testing
# <span style="font-size:25px;"> **Model:** Regression Analysis
# <span style="font-size:25px;"> **Train:** 18 datasets 
# <span style="font-size:25px;"> **Test:** 4 datasets
---
# <span style="font-sixze:25px;"> **Description:**
#  <span style="font-size:20px;"> * Importing required libraties.
#  <span style="font-size:20px;"> * Specifiing the inputs.
#  <span style="font-size:20px;"> * Loading all datasets and merging into 1 datadrame.
#  <span style="font-size:20px;"> * Separating dependent and independent variables.
#  <span style="font-size:20px;"> * Separating training and testing data.
#  <span style="font-size:20px;"> * Building regression model .
#  <span style="font-size:20px;"> * Fitting train data into model
#  <span style="font-size:20px;"> * Predicting the test data.
#  <span style="font-size:20px;"> * Calculating the error estimation
#  <span style="font-size:20px;"> * Visualizing the results
---

# <span style="font-size:25px;"> **Author:** Shashank P
---

In [ ]:
import xgboost as xgb
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,root_mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

import onnxruntime.quantization as quant
from skl2onnx.common.data_types import FloatTensorType

import xlsxwriter

In [ ]:

# Root directory containing all project-related data

## Input for folder path containing all thermal related data
mother_path = "D:\\2.ProjectWork\\1.TEX"

## Input for project /machine number
machine_number = "25TEX_006"

## Input for Phase number of the project
phase_number = "P04"

## Input for Data folder
model_folder ="10"

## Input for val files
val_files = [
    "25TEX_006_P03_04_251209.xlsx",
    "25TEX_006_P03_04_251210.xlsx",
#     "25TEX_006_P04_04_024.xlsx",
#     "25TEX_006_P04_04_025.xlsx",
#     "25TEX_006_P04_04_026.xlsx"
]

## Input for result folder (optional)
result_folder = "06"


## Input for result file (optional)
result_file = "25TEX_006_P04_06_021"
result_file = result_file + '.xlsx'


ind_list = [
"F1",
"F2",
"F3",
"F4" ,
# "F5",
"F6"
]

dep_list = ["D1"]

project_path = mother_path +"\\" + machine_number + "\\"+ machine_number + "_" + phase_number 
intial_fol_path = project_path + "\\" + machine_number + "_" + phase_number + "_" + model_folder
# val_fol_path = project_path + "\\" + machine_number + "_" + phase_number + "_" + val_folder
dest_fol_path =  project_path + "\\" + machine_number + "_" + phase_number + "_" + result_folder + "\\" + result_file



## Get the list of all files inside the input (model) folder
file_detials = os.listdir(intial_fol_path)

## Remove val file from the train files
train_files = [files for files in file_detials if files not in val_files]

## Initialize an empty list to store DataFrames
all_data_list = []

# Loop through each Excel file, read it into a DataFrame, and append it
for file in train_files:
    file_path = os.path.join(intial_fol_path,file)

    # Read Excel file into pandas DataFrame
    df = pd.read_excel(file_path)

    # Append DataFrame to the list
    all_data_list.append(df)

# Concatenate all DataFrames into a single dataset
all_data = pd.concat(all_data_list)



# Extract independent features (temperature sensors)
ind_data = [all_data[temp] for temp in ind_list]
x_train = pd.concat(ind_data, axis=1)



# Extract dependent feature (thermal deviation)
dep_data = [all_data[dev] for dev in dep_list]
y_train = pd.concat(dep_data, axis=1)



linear_model = LinearRegression()
linear_model.fit(x_train,y_train)
# dev_pred = linear_model.predict(temp_test_2)
# dev_error = dev_test - dev_pred


single_result_desp_list = []
single_result_list = []


r2_list = []
rmse_list = []
mae_list = []
mape_list = []




for i in range(len(val_files)):
    # val_files = val_files[i]
   

    val_data = []

    val_file_path = os.path.join(intial_fol_path,val_files[i])
    val_data_df = pd.read_excel(val_file_path)
    


    val_ind_data = []
    for temp in ind_list:
        val_ind_data.append(val_data_df[temp])
    val_x = val_independent = pd.concat(val_ind_data,axis=1)

   

    val_dep_data = []
    for dev in dep_list:
        val_dep_data.append(val_data_df[dev])
        val_dependent = pd.concat(val_dep_data,axis=1)
    val_y = val_dependent


    # #Predicting result for Single Observation

    val_prediction = linear_model.predict(val_x)
    val_error = val_y - val_prediction
   


    obs_band = np.max(val_y) - np.min(val_y)
    pred_band = np.max(val_prediction) - np.min(val_prediction)
    error_band = np.max(val_error) - np.min(val_error)


    r2_val = r2_score(val_y,val_prediction)
    r2_list.append(r2_val)
    r2_df = pd.DataFrame(r2_list)

    rmse_val = root_mean_squared_error(val_y,val_prediction)
    rmse_list.append(rmse_val)
    rmse_df = pd.DataFrame(rmse_list)

    mae_val = mean_absolute_error(val_y,val_prediction)
    mae_list.append(mae_val)
    mae_df = pd.DataFrame(mae_list)

    mape_val = mean_absolute_percentage_error(val_y,val_prediction)
    mape_list.append(mape_val)
    mape_df = pd.DataFrame(mape_list)





    
    single_result_desp_dict = {
        "file name" : [val_files[i]], 
        "X-dev band" : [obs_band],
        "X-pred band" : [pred_band], 
        "X-error band" : [error_band],
        "R_2" : [r2_val],
        "RMSE" : [rmse_val],
        "MAE" : [mae_val],
        "MAPE" : [mape_val]
        }
    single_result_desp_df = pd.DataFrame(single_result_desp_dict)
    single_result_desp_list.append(single_result_desp_df)


    single_result_data_df = pd.DataFrame()
    single_result_data_df["Iterations"] = np.linspace(1,len(val_y),len(val_y))
    single_result_data_df["Observed"] = val_y
    single_result_data_df["Prediction"] = val_prediction
    single_result_data_df["Error"] = val_error

    single_result_df = pd.concat([single_result_data_df,single_result_desp_df],axis = 1)

    single_result_list.append(single_result_df)


    




all_result_df = pd.concat(single_result_list).reset_index()
all_result_desp_df = pd.concat(single_result_desp_list).reset_index()


mean_dict = {
    "Parameters" : ["R2","RMSE","MAE","MAPE"],
    "Mean values" : [
        r2_df.mean().iloc[0],
        rmse_df.mean().iloc[0],
        mae_df.mean().iloc[0],
        mape_df.mean().iloc[0]
        ]
}
mean_df = pd.DataFrame(mean_dict)



final_result_desp = all_result_desp_df.join(mean_df)





dest_file = (dest_fol_path)
with pd.ExcelWriter(dest_file, engine='xlsxwriter') as writer:
    final_result_desp.to_excel(writer, sheet_name = "file_detials", index=False)
    for i, df in enumerate(single_result_list, start=1):
        sheet_name = f"00{i + 6}"   # Or use a custom name if needed
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        workbook  = writer.book
        worksheet = writer.sheets[sheet_name]

        # Create a chart object
        chart = workbook.add_chart({'type': 'scatter'})

        # Configure the chart from DataFrame columns
        max_row = len(df) + 1
        chart.add_series({
            'name':       f"='{sheet_name}'!$B$1",
            'categories': f"='{sheet_name}'!$A$2:$A${max_row}",
            'values':     f"='{sheet_name}'!$B$2:$B${max_row}",
        })
        chart.add_series({
            'name':       f"='{sheet_name}'!$C$1",
            'categories': f"='{sheet_name}'!$A$2:$A${max_row}",
            'values':     f"='{sheet_name}'!$C$2:$C${max_row}",
        })
        chart.add_series({
            'name':       f"='{sheet_name}'!$D$1",
            'categories': f"='{sheet_name}'!$A$2:$A${max_row}",
            'values':     f"='{sheet_name}'!$D$2:$D${max_row}",
        })
        # Add chart title and axis labels
        # chart.set_title({'name': 'Temperature vs Deviation'})
        chart.set_x_axis({
            'name': 'Time (sec)',
            'label_position': 'low',
            "major_gridlines": {"visible": False, "line": {"color": "#BFBFBF", "width": 0.75}},
            "minor_gridlines": {"visible": False},
            
            })
        chart.set_y_axis({
            'name': 'Deviation (µm)',
             "major_gridlines": {"visible": False, "line": {"color": "#BFBFBF", "width": 0.75}},
            "minor_gridlines": {"visible": False}
            })


        # Insert chart into the sheet
        worksheet.insert_chart(
            'E2', 
            chart,{
             'x_scale': 1.2,   # width scaling (1 = 100%)
            'y_scale': 1    # height scaling
        }
        )

        # 🎨 Add thick border and rounded corners
        chart.set_chartarea({
            'border': {'color': 'black', 'width': 2},  # thick border
            'fill':   {'color': '#F8F8F8'}               # soft background
        })




print("✅ Chart created successfully inside Excel file!")

In [ ]:


plt.plot(all_result_df["Observed"], label= 'Observed')
plt.plot(all_result_df["Prediction"], label= 'Prediction')
plt.xlabel('Iterations')
plt.title("Iterations vs Deviations")
plt.ylabel('Deviations')
plt.legend()
plt.show()


plt.plot(all_result_df["Error"],label='error') 
plt.xlabel('Iterations')
plt.ylabel('Error')
# plt.xlim(0,500)
# plt.ylim(0,-10)
plt.legend() 
plt.show() 

print(final_result_desp)